In [1]:
import pandas as pd
from matplotlib import pyplot
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statistics

In [2]:
pwd()

'/Users/julia/Desktop/Thesis/Code Repository'

In [472]:
project = pd.read_csv('projectnumber.csv', sep=';')

In [4]:
project

,Project Number,Legacy Project Number
0,30000000,Unspecified
1,30001202,Unspecified
2,30001203,Unspecified
3,30001204,Unspecified
4,30001205,Unspecified
...,...,...
65529,30082912,Unspecified
65530,30082914,Unspecified
65531,30082919,Unspecified
65532,30082920,Unspecified


In [492]:
df = pd.read_csv('portfolio.csv') #low_memory=False

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2,3,4,5,6,8,12,13,14,16,18,19,20,23,24,26,27,32,33,34,35,38,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,59,60,61,62,79,80,95,96,97,98,100,112,113,114,115,116,117,122,126,127,128,129,130,131,132,138,143,144,145,159,163,164,166,169,172,175,176,201,202) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df1 = pd.read_csv('result.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2,3,5,9,11,13,14,15,16,35,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
columnlist=df.columns.tolist()

In [11]:
print(columnlist)

['Fiscal Period Code', 'Project Number', 'Project Agreement Type', 'Project Id', 'Project Last Approved Forecast Date', 'Project Location', 'Project Manager Employee Number (Current)', 'Project Manager Name (Current)', 'Project Name', 'Project Status', 'Project Type', 'Ultimate Customer AUS Market Sector', 'Ultimate Customer Account Leader', 'Ultimate Customer NV Market Sector', 'Ultimate Customer Name', 'Ultimate Customer Number', 'Ultimate Customer Organization Size', 'Ultimate Parent AUS Market Sector', 'Ultimate Parent Account Leader', 'Ultimate Parent NV Market Sector', 'Ultimate Parent Name', 'Ultimate Parent Number', 'Ultimate Parent Organization Size', 'Sales Cloud Opportunity Number', 'SalesCloudClientNo', 'Contract Name', 'Contract Number', 'Contract Type Name', 'Region', 'City', 'Country', 'Country Name', 'CustGlobalServiceType', 'CustMarketSector', 'CustPrimaryMarketArea', 'Customer Number', 'Division Name', 'Parent Name', 'Parent Number', 'Portfolio Director Name', 'Portfo

In [8]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [9]:
len(df) #len(df1)

2179046

## We select the columns that are really revlevnat for our analysis

In [493]:
df1 = df[["Fiscal Period Code",
"Project Number",
"Project Id",
"Project Name",
"Health and Safety Status",
"project_date_create",
"Project Status",
"SalesCloudClientNo",
"Related Opportunity Project Number",
"Contract Type Name",
"Ultimate Customer AUS Market Sector",
"Division Short Name",
"location_2", 
"location_4",
"Region",
"City", 
"Country", 
"Country Name",
"Proj_CVP",
"Proj_Serv_Offering", 
"CVP Description",
"\ufeffCVP Code",
"primary_client_industry",
"Sales Cloud Opportunity Number",
"Funding Amount - ITC (Snapshot)",
"Backlog NR",
"Actual Direct Labor Cost - MTD",
"Actual Non-Labor Cost - MTD",
"Actual Gross Revenue - MTD",
"Actual Net Revenue - MTD",
"EBITA - MTD",
"Forecast Direct Labor Cost - ITC",
"Forecast Net Revenue - ITC",
"Order Intake (NR) - MTD",
"WBS1",
"bookings_jtd",
"interest_current",
"interest_jtd",
"interest_ytd",
"multiplier_budget",
"multiplier_current",
"multiplier_target",
"overhead_current", 
"overhead_rate",
"revenue_type", 
"AUS Mkt Sector Aggregate",
"Baseline NR", 
"Break Even Multiplier - Project",
"GBL Description",
"Legacy Project Number",
"Organization Size",
"Risk Profile",
"Sustainable Project", 
"interest_current",
"riskrating_project",
"Total WIP",
"Positive WIP",
"Total Aging",
]].copy()

In [332]:
len(df1)

2179046

In [494]:
df1.to_csv('subportfolio.csv')

In [495]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1['Project Number'] = df1['Project Number'].astype('str') 
df1['Project Number'] = df1['Project Number'].apply(lambda x: x.strip())
df1['Project Number'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [10]:
len(df1) #len(project), len(result)

2179046

In [21]:
df1.columns

Index(['Fiscal Period Code', 'Project Number', 'Project Id', 'Project Name',
       'Health and Safety Status', 'project_date_create', 'Project Status',
       'SalesCloudClientNo', 'Related Opportunity Project Number',
       'Contract Type Name', 'Ultimate Customer AUS Market Sector',
       'Division Short Name', 'location_2', 'location_4', 'Region', 'City',
       'Country', 'Country Name', 'Proj_CVP', 'Proj_Serv_Offering',
       'CVP Description', '﻿CVP Code', 'primary_client_industry',
       'Sales Cloud Opportunity Number', 'Funding Amount - ITC (Snapshot)',
       'Backlog NR', 'Actual Direct Labor Cost - MTD',
       'Actual Non-Labor Cost - MTD', 'Actual Gross Revenue - MTD',
       'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_t

## Let's replace legacy numbers with new project numbers based on info from dataframe

In [496]:
#replace all NaNs for df1
df1['Legacy Project Number'] = df1['Legacy Project Number'].astype('str') 
df1['Legacy Project Number']= df1['Legacy Project Number'].str.replace(r'^(NaN)$','Unspecified')
df1['Legacy Project Number']= df1['Legacy Project Number'].str.replace(r'^(nan)$','Unspecified')
df1['Project Number'] = df1['Project Number'].astype('str') 
df1['Project Number'] = df1['Project Number'].str.replace(r'^(NaN)$','Unspecified')
df1['Project Number'] = df1['Project Number'].str.replace(r'^(nan)$','Unspecified')

#replace all NaNs for legacy project file
project['Project Number'] = project['Project Number'].astype('str') 
project['Project Number'] = project['Project Number'].str.replace(r'^(NaN)$','Unspecified')
project['Project Number'] = project['Project Number'].str.replace(r'^(nan)$','Unspecified')
project['Legacy Project Number'] = project['Legacy Project Number'].astype('str') 
project['Legacy Project Number']= project['Legacy Project Number'].str.replace(r'^(NaN)$','Unspecified')
project['Legacy Project Number']= project['Legacy Project Number'].str.replace(r'^(nan)$','Unspecified')

In [497]:
legacydf =df1[df1['Legacy Project Number'] != 'Unspecified']
legacydf = legacydf[legacydf['Project Number'] != 'Unspecified']
len(legacydf)

377977

In [498]:
#Only use unique combinations
legacydf_pn = (legacydf.drop_duplicates(subset='Project Number', keep= 'first').reset_index(drop=True))
len(legacydf_pn)

20157

In [499]:
## There are no doubles! so let
df_count=legacydf_pn.groupby(['Legacy Project Number','Project Number']).size().reset_index().rename(columns={0:'count'})
df_count=df_count[df_count['count'] > 1]
df_count.sort_values('count', inplace=True, ascending=False)
df_count

,Legacy Project Number,Project Number,count


In [500]:
#subset df
leg = legacydf_pn[['Project Number', 'Legacy Project Number']].copy()

#make sure that the datatype is correct
leg['Project Number'] = leg['Project Number'].astype('str') 
leg['Legacy Project Number'] = leg['Legacy Project Number'].astype('str') 


#remove punctuation
leg['Legacy Project Number'] = leg['Legacy Project Number'].str.replace(r'.','')
leg['Project Number'] = leg['Project Number'].str.replace(r'.','')

#zip into dict
legacydict = dict(zip(leg['Legacy Project Number'], leg['Project Number']))

In [501]:
#Make sure that there is no punctuation in the original df as well

df1['Project Number'] = df1['Project Number'].astype('str')
df1['Project Number'] = df1['Project Number'].str.replace(r'.','')

# replace dict values in dataframe df1
df1['Project Number']=df1['Project Number'].map(legacydict).fillna(df1['Project Number'])

In [502]:
len(legacydict), len(df1)

(19606, 2179046)

In [503]:
## double check of an example  
df1[df1['Project Number'] == 'B00179270001-CSX Warm Springs, GA'].head() #007267880A04 #30001617 30055008,"B00179270001-CSX Warm Springs, GA"

,Fiscal Period Code,Project Number,Project Id,Project Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,GBL Description,Legacy Project Number,Organization Size,Risk Profile,Sustainable Project,interest_current,riskrating_project,Total WIP,Positive WIP,Total Aging


### Now we add the external list of project Numbers as well

In [129]:
project.describe()

,Project Number,Legacy Project Number
count,65534,65533
unique,65534,5026
top,30034996,Unspecified
freq,1,60301


In [504]:
#reduces the list of projects by about 63.000 enrties
legacy = project[project['Legacy Project Number'] != 'Unspecified']
legacy = legacy[legacy['Project Number'] != 'Unspecified']

legacy['Project Number'] = legacy['Project Number'].astype('str') 
legacy['Project Number'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [505]:
len(legacy), len(project)

(5232, 65534)

In [506]:
#We can see that there are more project numbers than legacy numbers
legacy['Legacy Project Number'].nunique(), legacy['Project Number'].nunique()

(5025, 5232)

## Preparing the project numbers list where we remove the unspecified

In [507]:
#So first we only keep unique project Numbers
legacy = (legacy.drop_duplicates(subset='Project Number', keep= 'first').reset_index(drop=True))
len(legacy)

5232

In [508]:
df_count=legacy.groupby(['Project Number', 'Legacy Project Number']).size().reset_index().rename(columns={0:'count'})
df_count=df_count[df_count['count'] != 1]
df_count.sort_values('count', inplace=True, ascending=False)
df_count

,Project Number,Legacy Project Number,count


In [162]:
## Remove punctuation from the legacy numbers
#leg['Legacy Project Number'] = leg['Legacy Project Number'].str.replace('[^\w\s]','')


## Select only the unique legacy numbers, then merge back with the project numbers, merge on project number

In [509]:
#make sure that the datatype is correct
legacy['Project Number'] = legacy['Project Number'].astype('str') 
legacy['Legacy Project Number'] = legacy['Legacy Project Number'].astype('str') 


#remove punctuation
legacy['Legacy Project Number'] = legacy['Legacy Project Number'].str.replace(r'.','')


#zip into dict
legacydict = dict(zip(legacy['Legacy Project Number'], legacy['Project Number']))

#Make sure that there is no punctuation in the original df as well

df1['Project Number'] = df1['Project Number'].astype('str')
df1['Project Number'] = df1['Project Number'].str.replace(r'.','')

# replace dict values in dataframe df1
df1['Project Number']=df1['Project Number'].map(legacydict).fillna(df1['Project Number'])

In [510]:
df1.to_csv('legacy_mapped.csv')
#df1 = pd.read_csv('subportfolio.csv')

### Project Name Mapping (as there are some with no Ids, but the same name)

In [511]:
del df
del df1

In [512]:
df = pd.read_csv('legacy_mapped.csv') 

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2,3,5,9,11,13,14,18,19,20,21,22,23,24,35,45,46,49,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [359]:
import re

In [360]:
len(df)

2179046

In [513]:
df['Project Number'] = df['Project Number'].astype('str') 
#df['Fiscal Period Code'] = pd.to_datetime(df['Fiscal Period Code'], format='%Y-%m') 
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [514]:
def repl(x):
    x = re.sub(r"[^\w\s]", "", str(x))      #Removes all punctuations
    x = re.sub(r"\d+", "", str(x))       #Replaces digits
    x = re.sub(r"\s+", " ", str(x))          #Removes double spaces with single space
    #x = x.strip().lower()                    #Trims the string
    return x

In [515]:
df['Cleaned Name'] = df['Project Name'].apply(repl)

In [583]:
#df[df['Cleaned Name']== 'TOS SN Honeywell DCS Support'] 

In [518]:
# obtain unique project numbers with their corresponding names in a df, and save this into a dictionary
# store the mapping in a new column and see whether there are differences later
#so now we are only mapping unique project numbers and names together

unique = (df.drop_duplicates(subset='Project Number', keep= 'last').reset_index(drop=True))

In [519]:
len(unique), df['Project Number'].nunique(), df['Cleaned Name'].nunique(), df['Project Name'].nunique()

(63693, 63693, 91202, 105465)

In [520]:
#df_nameswithmorethanonenumber=pd.DataFrame({'Project Name':df_nameswithmorethanonenumber.index, 'count':df_nameswithmorethanonenumber.values})
#unique came from only 2019+ data
df_combo=df.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('Project Name').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
dfdf_combo.reset_index(inplace=True)
dfdf_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)
dfdf_combo

,Project Name,count
9450,ACCRUED FEES,5094
27596,COST TRANSFER,5085
38503,DORMANT PROJECT EXP,4933
84707,REVENUE GROUP PROJECT,205
8,#XL_EVAL_ERROR#,56
...,...,...
10802,ARC00332.1000-GE HR 2019 NRD WILDLIFE OBSERVAT...,1
45410,FMC MacClennyFL_REM&Restoratio,1
10803,ARC01081.1000-GE UHR FP 2019 HHRA,1
45408,FMC MacClenny FL Assessment,1


#### The problem is that there are several names which have more than one project number - This is because either there is a notation error, or they actualy started several projects while the system was transitioned. We have to exclude those for the mapping phase, but they can probably still be used in the overall analysis as they have unique project numbers For now I remove the dots, and then after that map nan columns to project numbers based on project name.

In [521]:
df_toomanyprojects=dfdf_combo[(dfdf_combo['count'] != 2) & (dfdf_combo['count'] != 3)]
len(df_toomanyprojects) 
df_toomanyprojects['Project Name']

9450                                          ACCRUED FEES
27596                                        COST TRANSFER
38503                                  DORMANT PROJECT EXP
84707                                REVENUE GROUP PROJECT
8                                          #XL_EVAL_ERROR#
                               ...                        
10802    ARC00332.1000-GE HR 2019 NRD WILDLIFE OBSERVAT...
45410                       FMC MacClennyFL_REM&Restoratio
10803                    ARC01081.1000-GE UHR FP 2019 HHRA
45408                          FMC MacClenny FL Assessment
52732                 GWR000KB.6477-HI_306477_Honolulu_GWR
Name: Project Name, Length: 88523, dtype: object

In [522]:
#remove these names and create a new df1 or should I only be using df_1921 here? Here I just ignore all the values that have more than 7 different ids per name
df_mergebacklater = df[df['Project Name'].isin(df_toomanyprojects['Project Name'])]

In [523]:
#Whenever we create a subset, we need to make sure that the project numbers are still strings
#we work with those that have 2 or 3 as values
df1 = df[~df['Project Name'].isin(df_toomanyprojects['Project Name'])]

df1['Project Number'] = df1['Project Number'].astype('str') 
df1['Project Number']= df1['Project Number'].str.replace(r'.','') 

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### okey so we need projects names that currently have 1 or 2 project numbers after we drop unspecified, 
and only have one project number after 2019-07

In [524]:
#we wanna drop all NaN values
df2 = df1[~df1['Project Number'].str.contains("Unspecified")]
df2 = df2[~df2['Project Number'].str.contains("nan")]

In [525]:
len(df2), len(df1)

(126290, 362104)

In [526]:
df_combo=df2.groupby([ "Project Name", "Project Number"] ).size().to_frame(name = 'count').reset_index()
df_combo=df_combo.groupby('Project Name').count()
df_combo.pop('Project Number')


#This should be the names with more than two numbers
df_combo.reset_index(inplace=True)
df_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_combo

,Project Name,count
6046,FDOT - TWO7 - SR527 S Orange A,3
16472,WSIP - T&M,3
7234,GM Flint Env OS K2 Body Shop,3
8877,JLL BOA TX2-837 Bedford TX IAQ,3
6045,FDOT - TWO7 - SR525 S Orange A,3
...,...,...
5797,Eval. of Nutrient Impacts from Eastside,1
5798,Eval. of Sidestream Rerouting at M-L,1
5799,Evaluation of Dye Facility Loc,1
5800,Evanston Custer,1


In [440]:
#df_combo[(df_combo['count'] == 2)]

In [527]:
### only run the dict on df1 later, and then merge back with df_mergebacklater, otherwise we loose numbers
#only select the ones with 2 or 1 now. for those with 1, it means that they have a unique Id and unspecified. For those with 3, they might have a legacy ID and a new one
df3 = df_combo[(df_combo['count'] != 3)]

df_final = df1[df1['Project Name'].isin(df3['Project Name'])] 

#df2019 = df_final[df_final['Fiscal Period Code'] > 201906]
df_final = (df_final.sort_values('Fiscal Period Code').drop_duplicates(subset='Project Name', keep = 'last').reset_index(drop=True))
len(df_final)

16935

In [528]:
df_final[df_final['Cleaned Name']== 'TOS SN Honeywell DCS Support'] 

,Fiscal Period Code,Project Number,Project Id,Project Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,Legacy Project Number,Organization Size,Risk Profile,Sustainable Project,interest_current.1,riskrating_project,Total WIP,Positive WIP,Total Aging,Cleaned Name
11442,201906,30002921,MPICLIENT050990000080310,TOS SN-101 Honeywell DCS Support,NaN,NaN,A,10238715.0,NaN,Time and Material with cap,...,Unspecified,Local,NaN,NaN,25.34,2.0,NaN,NaN,NaN,TOS SN Honeywell DCS Support


In [285]:
len(df1), len(df_mergebacklater), len(df), df1['Project Number'].nunique(), df['Project Number'].nunique()
df[df['Legacy Project Number'] == 'OK0014760010']
df[df['Project Name'] == 'TX0021531801']

(1617746, 561300, 2179046, 48397, 63693)

In [543]:
df_final['Project Name'].nunique(), df_final['Project Number'].nunique()


(16935, 16928)

In [542]:
#df[df['Project Number'] == '30009752']

In [544]:
df_final = df_final.rename(columns={'Project Name': 'Project_Name', 'Project Number':'Project_Number'})
df = df.rename(columns={'Project Name': 'Project_Name', 'Project Number':'Project_Number'})

In [545]:
len(df1)

362104

In [547]:
### so now we will replace the unspceified and perhaps old numbers with the new ones
#make sure that the datatype is correct
#cannot be done with dictionary, becuase we want to match the name, and then fill in the Project number column
realData = pd.Series(data=df_final.Project_Number.values, index=df_final.Project_Name)

df['Project_Number'] = df.Project_Name.map(realData).fillna(df['Project_Number'])

In [548]:
len(df1), len(df)

(362104, 2179046)

In [552]:
#df[df['Project_Number'] == '30009752']

In [553]:
#df[df['Project_Name'] == "211411 - CHEVRON'S COST SHARE"]

In [554]:

df.to_csv('mapped_today.csv')

## Next we see whether we can map projects based on cleaned name

In [446]:
df = pd.read_csv('mapped_today.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (2,3,5,9,11,13,14,18,19,20,21,22,23,24,35,45,46,49,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [451]:
len(df)

2179046

In [447]:
df.columns

Index(['Unnamed: 0', 'Fiscal Period Code', 'Project_Number', 'Project Id',
       'Project_Name', 'Health and Safety Status', 'project_date_create',
       'Project Status', 'SalesCloudClientNo',
       'Related Opportunity Project Number', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Funding Amount - ITC (Snapshot)', 'Backlog NR',
       'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current',

In [555]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project_Number'] = df['Project_Number'].astype('str') 

In [588]:
#Cleaned names count, but first we need to map unique unclean names before we can map cleaned names
df_clean=df.groupby([ "Cleaned Name", "Project_Number"] ).size().to_frame(name = 'count').reset_index()
df_clean=df_clean.groupby('Cleaned Name').count()
df_clean.pop('Project_Number')


#This should be the names with more than two numbers
df_clean.reset_index(inplace=True)
df_clean.sort_values('count', 
                      inplace=True, 
                      ascending=False)
df_clean

,Cleaned Name,count
3832,ACCRUED FEES,5094
20372,COST TRANSFER,5085
30538,DORMANT PROJECT EXP,4933
72291,REVENUE GROUP PROJECT,207
41933,GPBPNAMMI,189
...,...,...
31224,DermPath Port Chester ESA,1
31222,Derby Main Street South NVCOG,1
31221,Derby Avenue West Haven Phase II ESA,1
31220,Derailment support Franklin,1


In [589]:
### Now we are going to replace all of those with unspecified with their old number
unspecifed_cleaned_names = df_clean[df_clean['count'] == 2]

#Extract those names again 
unspecifed_cleaned_names = df[df['Cleaned Name'].isin(unspecifed_cleaned_names['Cleaned Name'])] 

#Now this time remove the project numbers with NaN or unspecified
unspecifed_cleaned_names = unspecifed_cleaned_names[~unspecifed_cleaned_names['Project_Number'].str.contains("Unspecified")]
unspecifed_cleaned_names = unspecifed_cleaned_names[~unspecifed_cleaned_names['Project_Number'].str.contains("nan")]

In [592]:
#another count
df_clean=unspecifed_cleaned_names.groupby([ "Cleaned Name", "Project_Number"] ).size().to_frame(name = 'count').reset_index()
df_clean=df_clean.groupby('Cleaned Name').count()
df_clean.pop('Project_Number')


#This should be the names with more than two numbers
df_clean.reset_index(inplace=True)
df_clean.sort_values('count', 
                      inplace=True, 
                      ascending=False)

df_clean

,Cleaned Name,count
1096,DIVISION UST UPGRADES AND REPAIRS,2
1449,GUA TX Salcaja GWS,2
2418,SWP OMM East Point LS,2
1446,GUA TX Pelayo GWS,2
1445,GUA TX Miraflores GWS,2
...,...,...
1599,Hubbell Katy Tier II RY,1
1596,Hubbell Juarez COA,1
1594,Hubbell Houston Tier II RY,1
1588,Hollywood FL WWS,1


In [600]:
unspecifed_cleaned_names

,Fiscal Period Code,Project_Number,Project Id,Project_Name,Health and Safety Status,project_date_create,Project Status,SalesCloudClientNo,Related Opportunity Project Number,Contract Type Name,...,Legacy Project Number,Organization Size,Risk Profile,Sustainable Project,interest_current.1,riskrating_project,Total WIP,Positive WIP,Total Aging,Cleaned_Name
0,201901,CPEE16030002,ADVCLIENT5634,CP Mason City WWTP Construction Managem,NaN,NaN,D,10239682.0,NaN,Time and Material with cap,...,Unspecified,RKC,NaN,NaN,0.0,2.0,NaN,NaN,NaN,CP Mason City WWTP Construction Managem
1,201901,046130790000,ADVCLIENT10297,FY 2017 MDRRF Annual Report,NaN,NaN,D,10238812.0,NaN,Lump Sum,...,Unspecified,RKC,NaN,NaN,0.0,1.0,NaN,NaN,NaN,FY MDRRF Annual Report
2,201901,HT2121560018,LFRclient0215506240814127,Minute Men Cleaners 2018,NaN,NaN,D,10239554.0,NaN,Time and Material with cap,...,Unspecified,Local,NaN,NaN,0.0,1.0,NaN,NaN,NaN,Minute Men Cleaners
3,201901,TO077633CEI3,ADVCLIENT0821,MPC-CharlotteNC(E)-VRU,NaN,NaN,D,10238859.0,NaN,Time and Material with cap,...,Unspecified,RKC,NaN,NaN,0.0,2.0,NaN,NaN,NaN,MPCCharlotteNCEVRU
4,201901,CXPP1768PE00,ADVCLIENT3021,"SC0406 Florence, SC, Koppers Rd.",NaN,NaN,D,10239458.0,NaN,Time and Material with cap,...,Unspecified,RKC,NaN,NaN,0.0,2.0,NaN,NaN,NaN,SC Florence SC Koppers Rd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,202103,30003212,3.00001e+14,06614035.0000-CEG: Modeling for SP FY16,Plan Not Required,NaN,Closed-Open to Collect,NaN,NaN,Time and Material with cap,...,Unspecified,NaN,Medium,Unspecified,NaN,NaN,-13.26,0.0,NaN,CEG Modeling for SP FY
753,202103,30003219,3.00001e+14,06614047.0000-CEG: Citizens Reservoir,Plan Not Required,NaN,Closed,NaN,NaN,Time and Material with cap,...,06614047.0000,NaN,High,Unspecified,NaN,NaN,0.00,0.0,NaN,CEG Citizens Reservoir
754,202103,30003240,3.00001e+14,06753007.0000-Jefferson Co. 2017-2019 Sewer,Plan Not Required,NaN,Open/Active,NaN,NaN,Time and Material,...,06753007.0000,NaN,Low,Unspecified,NaN,NaN,2644.60,2644.6,NaN,Jefferson Co Sewer
755,202103,30003241,3.00001e+14,06753008.0000-Jefferson Co. 2017-2019 Water,Plan Not Required,NaN,Open/Active,NaN,NaN,Time and Material,...,06753008.0000,NaN,Low,Unspecified,NaN,NaN,5903.60,5903.6,NaN,Jefferson Co Water


In [593]:
#Now the cleaned count can only be equal to 1
unspecifed_cleaned_names = df_clean[df_clean['count'] == 1]


#Then we need to extract the whole rows again to turn it into a dictionary
unspecifed_cleaned_names = df[df['Cleaned Name'].isin(unspecifed_cleaned_names['Cleaned Name'])] 


unspecifed_cleaned_names = (unspecifed_cleaned_names.drop_duplicates(subset='Cleaned Name', keep= 'last').reset_index(drop=True))
len(unspecifed_cleaned_names)

757

In [595]:
unspecifed_cleaned_names = unspecifed_cleaned_names.rename(columns={'Cleaned Name': 'Cleaned_Name'})
df = df.rename(columns={'Cleaned Name': 'Cleaned_Name'})

In [596]:
len(df)

2179046

In [597]:
## create a script where we match the same cleaned names and 

realData = pd.Series(data=unspecifed_cleaned_names.Project_Number.values, index=unspecifed_cleaned_names.Cleaned_Name)

df['Project_Number'] = df.Cleaned_Name.map(realData).fillna(df['Project_Number'])

In [612]:
#df[df['Cleaned_Name'] == 'CEG Citizens Reservoir']

In [614]:
df.columns

Index(['Fiscal Period Code', 'Project_Number', 'Project Id', 'Project_Name',
       'Health and Safety Status', 'project_date_create', 'Project Status',
       'SalesCloudClientNo', 'Related Opportunity Project Number',
       'Contract Type Name', 'Ultimate Customer AUS Market Sector',
       'Division Short Name', 'location_2', 'location_4', 'Region', 'City',
       'Country', 'Country Name', 'Proj_CVP', 'Proj_Serv_Offering',
       'CVP Description', '﻿CVP Code', 'primary_client_industry',
       'Sales Cloud Opportunity Number', 'Funding Amount - ITC (Snapshot)',
       'Backlog NR', 'Actual Direct Labor Cost - MTD',
       'Actual Non-Labor Cost - MTD', 'Actual Gross Revenue - MTD',
       'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_t

In [680]:
df['Project Status'].unique()

array(['Open/Active', 'On Hold - Inactive', 'Closed',
       'Closed-Open to Collect', 'Final Closure Requested', 'New',
       'New - Pending Approval', 'Rejected', 'Unspecified', 'Manual_Adj'],
      dtype=object)

In [648]:
import numpy as np

In [677]:
df['Project Status']= df['Project Status'].str.replace(r'^(A)$','Open/Active')
df['Project Status']= df['Project Status'].str.replace(r'^(D)$','On Hold - Inactive')
df['Project Status']= df['Project Status'].str.replace(r'^(I)$','On Hold - Inactive')
df['Project Status'].fillna('Unspecified', inplace=True)
df['Project Status']= df['Project Status'].str.replace(r'^(Not Applicable)$', 'Unspecified')
## New - pending, rejected, manual adjust are ignored for forecasting

In [683]:
df['Contract Type Name'].unique()

array(['Time and Material', 'Lump Sum', 'Cost plus Fixed Fee', 'D', 'N',
       'R', 'H', 'Unspecified', 'Time and Material with cap',
       'Revenue group', 'Lump Sum with IFRS'], dtype=object)

In [682]:
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(T)$', 'Time and Material')
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(L)$', 'Lump Sum')
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(C)$', 'Cost plus Fixed Fee')
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(M)$', 'Lump Sum')
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(0)$', 'Unspecified')
df['Contract Type Name']= df['Contract Type Name'].str.replace(r'^(Not Applicable)$', 'Unspecified')
df['Contract Type Name'].fillna('Unspecified', inplace=True)

In [627]:
df['Health and Safety Status']= df['Health and Safety Status'].str.replace(r'^(nan)$', 'Unspecified')

In [628]:
df['Division Short Name'].unique()

array(['Environment', 'Water', 'Enabling Function', 'Buildings',
       'Infrastructure'], dtype=object)

In [633]:
df['Country'].unique()

array(['United States', 'Germany', 'Israel', nan, 'United Kingdom',
       'Argentina', 'Italy', 'Puerto Rico', 'Brazil', 'Panama',
       'El Salvador', 'Bahamas', 'Nicaragua', 'Guatemala', 'Ecuador',
       'Honduras', 'France', 'Canada', 'India', 'Australia',
       'Netherlands', 'Indonesia', 'Malaysia', 'Turkey', 'Peru',
       'Belgium', 'Guam', 'Aruba', 'Slovakia', 'West Indies', 'Norway',
       'Barbados', 'Virgin Islands (US)', 'Mexico', 'Dominican Republic',
       'Czech Republic', 'China', 'Uruguay', 'Hong Kong', 'Japan',
       'Korea, Republic of', 'United Arab Emirates', 'Chile', 'Taiwan',
       'Switzerland', 'South Africa', 'Portugal', 'Hungary', 'Qatar',
       'Egypt', 'Singapore', 'Philippines', 'Colombia', 'Denmark',
       'Saudi Arabia', 'Haiti', 'Ireland', 'Russian Federation',
       'Cayman Islands', 'Grenada', 'Thailand', 'Sweden', 'Spain',
       'Vanuatu', 'Trinidad and Tobago', 'Viet Nam', 'Finland', 'Austria',
       'Afghanistan', 'Bermuda', 'New Zeala

In [684]:
df['Country']= df['Country'].str.replace(r'^(Virgin Islands, U.S.)$', 'Virgin Islands (US)')
df['Country']= df['Country'].str.replace(r'^(United States Minor Outlying Islands)$', 'United States')
df['Country']= df['Country'].str.replace(r'^(US)$', 'United States')
df['Country']= df['Country'].str.replace(r'^(Taiwan, Province of China)$', 'Taiwan')
df['Country']= df['Country'].str.replace(r'^(Taiwan, Republic of China)$', 'Taiwan')
df['Country']= df['Country'].str.replace(r'^(Republic of Ireland)$', 'Ireland')

df['Country'].fillna('Unspecified', inplace=True)

In [634]:
#Clean up regions before usage
df['Region'].unique()

array(['NY', 'VA', 'OH', 'PA', 'NJ', 'MD', nan, 'CA', 'AZ', 'TX', 'CT',
       'KY', 'CO', 'FL', 'DC', 'AL', 'RI', 'PR', 'IN', 'NC', 'MA', 'DE',
       'MI', 'WV', 'WA', 'IL', 'NM', 'OK', 'LA', 'TN', 'BERKS', 'AK',
       'SC', 'GA', 'UT', 'MT', 'NH', 'MO', 'KS', 'OR', 'HI', 'NV', 'AB',
       'VT', 'WI', 'SD', 'WY', 'ID', 'MN', 'NT', 'ON', 'IA', 'NO', 'SK',
       'AR', 'MB', 'ND', 'GU', 'QC', 'MS', 'LONDON', 'BC', 'NE', 'ME',
       'NU', 'NS', 'VI', 'SUFF', 'LINCS', 'PE', 'TYNE', 'CHES', 'NL',
       'VIC', 'CAMBS', 'CORK', 'MAH', 'QLD', 'ACT', 'SX', 'NSW', 'GLOS',
       'LANCS', 'STAFFS', 'ABER', 'California, US', 'New Jersey, US',
       'Ohio, US', 'New Mexico, US', 'New York, US', 'Pennsylvania, US',
       'Florida, US', 'Louisiana, US', 'Arkansas, US', 'Wisconsin, US',
       'Virginia, US', 'Missouri, US', 'Kentucky, US',
       'District of Columbia, US', 'Connecticut, US', 'Massachusetts, US',
       'Minnesota, US', 'Mississippi, US', 'Georgia, US', 'Delaware, US',
      

In [635]:
#Country Name not important
df['Country Name'].unique()

array(['United States', nan, 'Canada'], dtype=object)

In [636]:
df['Proj_CVP'].unique()

array(['Strategic Environmental Consulting', 'Conveyance',
       'Water Supply and Treatment', 'Water Management',
       'Site Evaluation and Restoration', 'Water for Industry',
       'Environmental Construction Services', 'Environmental Planning',
       nan, 'Commercially Led Program Management', 'Business Advisory',
       'GRIP', 'Rail & Urban Transport',
       'Ports and Industrial Infrastructure Solutions',
       'Mining Solutions', 'Overhead Project', 'Highways & ITS',
       'Master Planning & Sustainable Urban Development',
       'Contract Solutions', 'Airport Solutions',
       'Performance Driven Design', 'N/A - NOT APPLICABLE',
       'Program Management (PM)', 'Cost Management (CM)',
       'Performance Driven Engineering (PDE)',
       'Performance Driven Architecture (PDA)'], dtype=object)

In [637]:
df['Proj_Serv_Offering'].unique()

array(['EN - Due Diligence/M&A Support', 'WR - Drinking Water Supply',
       'WR - Information Technology', 'WR - Wastewater Treatment',
       'WR - Program Management', 'WR - Drinking Water Treatment',
       'EN - Remediation Design & Engineering',
       'WR - Stormwater/Urban Drainage',
       'EN - Environmental Regulatory Compliance & Auditing',
       'EN - Hydrogeology/Modeling', 'EN - Air & Noise Services',
       'WR - Integrated Water Resource Mgmt',
       'WR - Management Consulting',
       'EN - Remediation Technical Consulting',
       'WR - Water Construction Mgmt', 'WR - Wastewater Collection',
       'EN - Risk Assessment Management & Regulatory Strategies',
       'EN - Environmental Planning', 'Industrial Design & Engineering',
       'WR - Solid Waste', 'EN - Non-Financial Reporting Services/EMIS',
       'WR - Financial Consulting', 'EN - D4',
       'EN - Hazardous Building Materials (Asbestos & Lead)',
       'EN - Earthwork & Sediment Services', 'WR - Water 

In [685]:
df['CVP Description']= df['CVP Description'].str.replace(r'^(N/A - NOT APPLICABLE)$', 'Unspecified')
df['CVP Description'].fillna('Unspecified', inplace=True)
df['CVP Description'].unique()

array(['Unspecified', 'Strategic Environmental Consulting',
       'Water Supply and Treatment', 'Site Evaluation and Restoration',
       'Environmental Planning', 'Conveyance', 'Water Management',
       'Water for Industry', 'Business Advisory', 'Mining Solutions',
       'Environmental Construction Services',
       'Highways and ITS Solutions', 'Contract Solutions',
       'Rail & Urban Transport Solutions',
       'Ports and Industrial Infrastructure Solutions',
       'Overhead Project',
       'Master Planning & Sustainable Urban Development',
       'Project and Program Management', 'GRIP',
       'Cost and Commercial Management', 'Airport Solutions',
       'Performance Driven Engineering',
       'Performance Driven Architecture', 'Default'], dtype=object)

In [686]:
df['primary_client_industry'].fillna('Unspecified', inplace=True)
df['primary_client_industry'].unique()

array(['CP - Chem/Pharma', 'MU - Municipal', 'ST - State/Provinces',
       'OG - Oil & Gas', 'CONG - Conglomerate',
       'WC - Water Companies/Utilities', 'DOD - Dept of Defense',
       'NP - Non-Profit', 'DOT - State Transportation', 'FED - Federal',
       'IC - Arcadis Intercompany', 'CR - Consumer & Retail',
       'BLD - Building Materials & General Manufacturing',
       'MM - Mining/Metals', 'PC - Power Companies/Utilities',
       'PA - Pulp/Paper',
       'EE - Engineering, Environmental & Construction Management',
       'PPA - Ports/Public Authority', 'TC - Telecom Utilities',
       'FIN - Financial', 'CT - Contractors',
       'F&BP - Food & Bev Producers', 'IN - Internal',
       'CD - Commercial Developers', 'RL - Freight Rail',
       'LOG - Logistics', 'EL - Technology, Telecomm & Electronics',
       'AU - Automotive', 'AE - Aerospace', 'INS - Insurance',
       'PvtEd - Private Education', 'PubEd - Public Education',
       'DC - Digital & Creative Industries', '

In [689]:
df['Organization Size'].fillna('Unspecified', inplace=True)
df['Organization Size'].unique()

array(['RKC', 'Local', 'GKC', 'Key Global', 'Key Regional', 'Unspecified',
       'Key Local'], dtype=object)

In [691]:
df['Risk Profile'].fillna('Unspecified', inplace=True)
df['Risk Profile'].unique()

array(['Unspecified', 'Low', 'Medium', 'High'], dtype=object)

In [693]:
#Can probably not be used
df['Sustainable Project'].fillna('Unspecified', inplace=True)
df['Sustainable Project'].unique()

array(['Unspecified', 'Level 1'], dtype=object)

In [694]:
df['interest_current.1'].unique()

array([ -23.04,    3.19,    1.75, ..., -638.74,  638.12,     nan])

In [695]:
df['riskrating_project'].unique()

array([nan,  1.,  2.,  3.,  5.,  4.,  0.])

In [696]:
df['Total WIP'].unique()

array([     nan,     0.  ,  2879.2 , ..., 51525.05,    85.03, 10766.14])

In [697]:
df['Positive WIP'].unique()

array([     nan,     0.  ,  2879.2 , ..., 51525.05,    85.03, 10766.14])

In [698]:
#column not important
df['Total Aging'].unique()

array([nan])

In [699]:
df.to_csv('mapped.csv')